# IW+ integration for regionalized impact categories with generalized beta distributions

Generalized beta distribution needs to be added in BW2. See corresponding notebook.

In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays

In [2]:
bw.projects.set_current('iw_integration')

## Creating the impact categories with uncertainty

Loading CFs with uncertainty information for csv file

In [3]:
path_to_input_csv=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Files\All IC regionalized_with uncertainty.csv'
iw_olca_uncert = csv.DictReader(open(path_to_input_csv, "r"), delimiter=";")
iw_olca_uncert_list=[cf for cf in iw_olca_uncert]
iw_olca_uncert_dict={(cf['impact_cat_name'],cf['ef_UUID']):cf for cf in iw_olca_uncert_list}
iw_olca_uncert_dict

{('Freshwater acidification, GLO, with uncert',
  '003697f1-54e9-409e-8179-23a95e99e4d7'): {'CF_formula': '0.799194406',
  'CF_per_unit_name': 'kg',
  'CF_unit': 'PDF.m2.yr',
  'IW_Sample_CF_Max': '93.83769727',
  'IW_Sample_CF_Mean': '0.799247818',
  'IW_Sample_CF_Min': '0.002406182',
  'IW_Sample_CF_Std dev': '1.35155162',
  'IW_Sample_CF_Variance': '1.826691782',
  'IW_ef': 'NOx',
  'Stoechiometric_ratio': '1',
  'beta_param_alpha': '0.336155493',
  'beta_param_beta': '39.24918627',
  'beta_param_max': '93.83769727',
  'beta_param_min': '0.002406182',
  'ef_UUID': '003697f1-54e9-409e-8179-23a95e99e4d7',
  'flow_category_subcompart': 'low population density',
  'flow_name': 'Nitrogen dioxide',
  'impact_cat_name': 'Freshwater acidification, GLO, with uncert',
  'impact_method_name': 'IMPACTWorld+ - Endpoint - only with spatial variability',
  'uncert_param1': '0',
  'uncert_param2': '0.799194406',
  'uncert_param3': '93.83769727',
  'uncert_type_name': 'triangle'},
 ('Freshwater acid

### Creating uncertainty dictionaries for each CFs.

The priority order for the choice of distribution is :
1- Generalized Beta distribution if alpha and beta positive or No distribution if specified
2- if alpha and beta negative, Triangular distribution with moment method
3- if mode_triangular<minimum, Bounded Normal with moment method (no problem with negative CFs)

In [4]:
def uncert_dict_triang_moment_method(sample_min,sample_mean,sample_max):
    
    mode = 3 * sample_mean - sample_min - sample_max
    
    if mode < sample_min:
        uncert_dict={'amount': sample_mean,
                'uncertainty type': stats_arrays.NoUncertainty.id};
    else:
        uncert_dict={'amount': sample_mean,
                'loc': mode,
                'maximum': sample_max,
                'minimum': sample_min,
                'uncertainty type': stats_arrays.TriangularUncertainty.id};
        
    return uncert_dict;
    

In [5]:
from stats_arrays import MCRandomNumberGenerator, UncertaintyBase


def test_distrib(ID,uncert_dict):
    params_distrib = UncertaintyBase.from_dicts(uncert_dict)
    mcrng_distrib = MCRandomNumberGenerator(params_distrib)
    try:
        mcrng_distrib.next()
        return;
    except:
        tuple_error=(ID,uncert_dict)
        return tuple_error;

    
    

In [6]:
iw_uncert_dict_beta={}
iw_uncert_beta_distrib={}
iw_uncert_none_distrib={}
iw_uncert_triang_distrib={}
iw_uncert_normal_distrib={}

list_error_generalized_beta=[]
list_error_triang=[]
list_error_normal=[]

i=0

for cf in iw_olca_uncert_dict:
    
    i=i+1
    
    amount=float(iw_olca_uncert_dict[cf]['CF_formula'])
    uncert_type=iw_olca_uncert_dict[cf]['uncert_type_name']
    
    if uncert_type.lower()=="none":
        uncert_dict=amount
        iw_uncert_dict_beta[cf]=uncert_dict
        iw_uncert_none_distrib[cf]=uncert_dict
        continue
    else:        
        alpha=float(iw_olca_uncert_dict[cf]['beta_param_alpha'])
        beta=float(iw_olca_uncert_dict[cf]['beta_param_beta'])
        minimum=float(iw_olca_uncert_dict[cf]['beta_param_min'])
        maximum=float(iw_olca_uncert_dict[cf]['beta_param_max'])
        mean=float(iw_olca_uncert_dict[cf]['IW_Sample_CF_Mean'])
        variance=float(iw_olca_uncert_dict[cf]['IW_Sample_CF_Variance'])
        mode_triang = 3 * mean - minimum - maximum
    
    if alpha>0 and beta>0:
        uncert_dict={'amount': amount,
                          'loc': alpha, 
                          'shape': beta, 
                          'maximum': maximum,
                          'minimum': minimum,
                          'uncertainty type': stats_arrays.GeneralizedBetaUncertainty.id,'uncertainty_type': stats_arrays.GeneralizedBetaUncertainty.id}
        iw_uncert_dict_beta[cf]=uncert_dict
        iw_uncert_beta_distrib[cf]=uncert_dict
        if test_distrib(cf,uncert_dict)!= None:
            list_error_generalized_beta.append(test_distrib(cf,uncert_dict))
        continue
     
    elif mode_triang >= minimum:
        uncert_dict=uncert_dict_triang_moment_method(minimum,mean,maximum)
        iw_uncert_dict_beta[cf]=uncert_dict
        iw_uncert_triang_distrib[cf]=uncert_dict
        if test_distrib(cf,uncert_dict)!= None:
            list_error_triang.append(test_distrib(cf,uncert_dict))
        continue
    
    else:
        uncert_dict={'amount': amount,
                          'loc': mean, 
                          'scale': np.sqrt(np.abs(variance)), 
                          'maximum': maximum,
                          'minimum': minimum,
                          'uncertainty type': stats_arrays.NormalUncertainty.id}
        iw_uncert_dict_beta[cf]=uncert_dict
        iw_uncert_normal_distrib[cf]=uncert_dict
        if test_distrib(cf,uncert_dict)!= None:
            list_error_normal.append(test_distrib(cf,uncert_dict))

iw_uncert_dict_beta

{('Freshwater acidification, GLO, with uncert',
  '003697f1-54e9-409e-8179-23a95e99e4d7'): {'amount': 0.799194406,
  'loc': 0.336155493,
  'maximum': 93.83769727,
  'minimum': 0.002406182,
  'shape': 39.24918627,
  'uncertainty type': 13,
  'uncertainty_type': 13},
 ('Freshwater acidification, GLO, with uncert',
  '006aa3f7-59ba-450f-aa45-a2b2d1752647'): {'amount': 1.591727031,
  'loc': 0.05646066,
  'maximum': 146.2778145,
  'minimum': 0.001710379,
  'shape': 5.137727289,
  'uncertainty type': 13,
  'uncertainty_type': 13},
 ('Freshwater acidification, GLO, with uncert',
  '08a91e70-3ddc-11dd-954d-0050c2490048'): {'amount': 1.746893022,
  'loc': 0.074258037,
  'maximum': 1556.719119,
  'minimum': 0.000401678,
  'shape': 66.11494813,
  'uncertainty type': 13,
  'uncertainty_type': 13},
 ('Freshwater acidification, GLO, with uncert',
  '08a91e70-3ddc-11dd-96ee-0050c2490048'): {'amount': 0.799194406,
  'loc': 0.336155493,
  'maximum': 93.83769727,
  'minimum': 0.002406182,
  'shape': 39.

In [7]:
len(list_error_generalized_beta),len(iw_uncert_beta_distrib)

(0, 818)

In [8]:
len(list_error_triang),len(iw_uncert_triang_distrib)

(0, 0)

In [9]:
len(list_error_normal),len(iw_uncert_normal_distrib)

(0, 0)

In [10]:
len(iw_uncert_dict_beta), len(iw_uncert_none_distrib)

(997, 179)

No errors

In [11]:
list_error_generalized_beta

[]

In [11]:
ic_errors_all=[error[0][0] for error in list_error_generalized_beta]
ic_errors=set(ic_errors_all)
ic_errors

{'Land occupation, biodiversity, GLO, with uncert',
 'Land transformation, biodiversity, GLO, with uncert',
 'Marine eutrophication, GLO, with uncert',
 'Water availability, human health, GLO, with uncert'}

In [13]:
cf_errors_all=[error[0] for error in list_error_generalized_beta]
cf_errors_all

[('Water availability, human health, GLO, with uncert',
  '995f3484-64f1-4d38-bc97-5364463e02b0'),
 ('Land transformation, biodiversity, GLO, with uncert',
  'c3eff967-20d8-4bcd-bd26-2dc2592a37b9'),
 ('Water availability, human health, GLO, with uncert',
  'e86c6358-1283-31b1-af5d-6bf05fc58ff8'),
 ('Water availability, human health, GLO, with uncert',
  'eba9b309-7926-37da-a4c5-0d4c2ef894f9'),
 ('Land transformation, biodiversity, GLO, with uncert',
  '891b2f36-c1cb-495a-9f27-be7b583eaf1b'),
 ('Land transformation, biodiversity, GLO, with uncert',
  'a6888cde-b33b-4bec-9e07-c6f3e6932f24'),
 ('Land occupation, biodiversity, GLO, with uncert',
  '97c598f0-4d1d-4f9f-906e-097347168153'),
 ('Water availability, human health, GLO, with uncert',
  '5d8950af-bf5f-3373-b75e-441c1b5d3743'),
 ('Land occupation, biodiversity, GLO, with uncert',
  '062a6faf-b1a5-4a6a-aa02-47ae3ec566a8'),
 ('Land transformation, biodiversity, GLO, with uncert',
  '46b8b2fd-eb2a-413d-bc39-e7dc18a420ef'),
 ('Water ava

In [21]:
valid_or_not={}

for cf in cf_errors_all:
    uncert_dict=iw_uncert_dict_beta[cf]
    uncert=stats_arrays.UncertaintyBase.from_dicts(uncert_dict)
    
    if stats_arrays.GeneralizedBetaUncertainty.check_bounds_reasonableness(uncert)!= None:
        valid_or_not[cf]=stats_arrays.GeneralizedBetaUncertainty.check_bounds_reasonableness(uncert)
    
valid_or_not  

{}

Bounds seem to be valid...

In [29]:
cdf={}

for cf in cf_errors_all:
    uncert_dict=iw_uncert_dict_beta[cf]
    uncert=stats_arrays.UncertaintyBase.from_dicts(uncert_dict)
    cdf[cf]=stats_arrays.GeneralizedBetaUncertainty.cdf(uncert,(uncert['minimum']-uncert['maximum'])/2)
    
cdf

{('Land occupation, biodiversity, GLO, with uncert',
  '012ee35c-c4e1-4759-9f62-e1a7a8504f8c'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '062a6faf-b1a5-4a6a-aa02-47ae3ec566a8'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '080bf4c6-fab3-4e21-88c7-20b435aa42e8'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '0a03fe20-343b-42d4-87b5-1300cfd47a7a'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '135816ab-f119-4a64-b608-a61cb788af7d'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '14811cde-0ef0-4327-ad94-4ee232f7a462'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '14b92328-e08d-4fac-b16a-da5dcd0e5627'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '19f84b2e-e6ff-4351-ba3a-8b650fc20d14'): array([[ 0.]]),
 ('Land occupation, biodiversity, GLO, with uncert',
  '1ae03e9f-9f0b-4f9e-8440-9c3d4f1b1868'): array([[

In [33]:
data=2
min_array=3
max_array=10

(data < min_array) + (data > max_array)

1

In [34]:
data[1]

TypeError: 'int' object is not subscriptable

In [ ]:
iw_uncert_none_distrib

In [39]:
list_ic_none=[]

for cf in iw_uncert_none_distrib:
    list_ic_none=list(set(list_ic_none)|set([cf[0]]))
    
list_ic_none

['Land transformation, biodiversity, GLO, with uncert',
 'Marine eutrophication, GLO, with uncert',
 'Water availability, human health, GLO, with uncert',
 'Land occupation, biodiversity, GLO, with uncert',
 'Freshwater eutrophication, GLO, with uncert']

None distributions OK

In [17]:
iw_uncert_triang_distrib

{('Land occupation, biodiversity, GLO, with uncert',
  '0d6e367d-0c4c-4f6d-9bff-9b370a182ff1'): {'amount': 0.632603085,
  'loc': 0.5978092549999998,
  'maximum': 0.76,
  'minimum': 0.54,
  'uncertainty type': 5},
 ('Land occupation, biodiversity, GLO, with uncert',
  '12501edd-e004-40be-bf07-01d7f4c32ea8'): {'amount': 0.632603085,
  'loc': 0.5978092549999998,
  'maximum': 0.76,
  'minimum': 0.54,
  'uncertainty type': 5},
 ('Land occupation, biodiversity, GLO, with uncert',
  '16df4fdf-06a1-4c84-bd49-b933d5bd2714'): {'amount': 0.632603085,
  'loc': 0.5978092549999998,
  'maximum': 0.76,
  'minimum': 0.54,
  'uncertainty type': 5},
 ('Land occupation, biodiversity, GLO, with uncert',
  '1b0a8570-eab4-46c2-9b67-c9b918e75676'): {'amount': 0.632603085,
  'loc': 0.5978092549999998,
  'maximum': 0.76,
  'minimum': 0.54,
  'uncertainty type': 5},
 ('Land occupation, biodiversity, GLO, with uncert',
  '1f99f0be-f142-4a18-a7cd-209bf22218c8'): {'amount': 0.632603085,
  'loc': 0.5978092549999998,

In [20]:
valid_or_not={}

for cf in iw_uncert_triang_distrib:
    uncert_dict=iw_uncert_triang_distrib[cf]
    uncert=stats_arrays.UncertaintyBase.from_dicts(uncert_dict)
    valid_or_not[cf]=stats_arrays.TriangularUncertainty.check_bounds_reasonableness(uncert)
    
valid_or_not  

{('Land occupation, biodiversity, GLO, with uncert',
  '0d6e367d-0c4c-4f6d-9bff-9b370a182ff1'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '12501edd-e004-40be-bf07-01d7f4c32ea8'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '16df4fdf-06a1-4c84-bd49-b933d5bd2714'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '1b0a8570-eab4-46c2-9b67-c9b918e75676'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '1f99f0be-f142-4a18-a7cd-209bf22218c8'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '2ffde7a9-81d0-4113-a519-3196e5886679'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '33817d31-6212-44bb-99b0-df1426ae92e5'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '35f58560-2ce4-4213-9984-616a292e63ea'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '3fa694bc-8c9e-406d-b7aa-f2e50d8511b8'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '4497ab6d-6666-43d1-abfd-56

Triangle distributions seems to be OK --> only land occupation as triangle

In [8]:
iw_uncert_normal_distrib

{('Land occupation, biodiversity, GLO, with uncert',
  '16835899-c22f-4e78-8076-0d4d340c6f60'): {'amount': 0.074625861,
  'loc': 0.074625861,
  'maximum': 0.42,
  'minimum': 0.02,
  'scale': 0.13735486886164611,
  'uncertainty type': 3},
 ('Land occupation, biodiversity, GLO, with uncert',
  '18636f13-f552-4136-a353-3b5a8e5f87d1'): {'amount': 0.074625861,
  'loc': 0.074625861,
  'maximum': 0.42,
  'minimum': 0.02,
  'scale': 0.13735486886164611,
  'uncertainty type': 3},
 ('Land occupation, biodiversity, GLO, with uncert',
  '1896b498-8d13-4f58-8c17-21fe57740158'): {'amount': 0.074625861,
  'loc': 0.074625861,
  'maximum': 0.42,
  'minimum': 0.02,
  'scale': 0.13735486886164611,
  'uncertainty type': 3},
 ('Land occupation, biodiversity, GLO, with uncert',
  '1a01977a-d170-4ac1-86df-aae792cbeb8f'): {'amount': 0.074625861,
  'loc': 0.074625861,
  'maximum': 0.42,
  'minimum': 0.02,
  'scale': 0.13735486886164611,
  'uncertainty type': 3},
 ('Land occupation, biodiversity, GLO, with unce

In [21]:
valid_or_not_normal={}

for cf in iw_uncert_normal_distrib:
    uncert_dict=iw_uncert_normal_distrib[cf]
    uncert=stats_arrays.UncertaintyBase.from_dicts(uncert_dict)
    valid_or_not_normal[cf]=stats_arrays.NormalUncertainty.check_bounds_reasonableness(uncert)
    
valid_or_not_normal

{('Land occupation, biodiversity, GLO, with uncert',
  '16835899-c22f-4e78-8076-0d4d340c6f60'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '18636f13-f552-4136-a353-3b5a8e5f87d1'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '1896b498-8d13-4f58-8c17-21fe57740158'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '1a01977a-d170-4ac1-86df-aae792cbeb8f'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '1c359785-3e70-4e0b-be1c-6258f9f20708'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '21baa1af-e8af-425b-a127-91629f718508'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '2997b0b4-f6ec-4df0-9481-4bdab4e86507'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '3295a31d-855c-4ed1-ace4-e81d0a7f04c6'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '3afbc2d8-1b51-4d43-87a7-e68291c1e999'): None,
 ('Land occupation, biodiversity, GLO, with uncert',
  '5db2250f-278c-4b47-8c45-56

Normal distributions seems to be OK --> only land occupation as normal

In [23]:
valid_or_not_beta={}

for cf in iw_uncert_beta_distrib:
    uncert_dict=iw_uncert_beta_distrib[cf]
    uncert=stats_arrays.UncertaintyBase.from_dicts(uncert_dict)
    valid_or_not_normal[cf]=stats_arrays.GeneralizedBetaUncertainty.check_bounds_reasonableness(uncert)
    
not_valid_beta=[valid for valid in valid_or_not_beta if valid != None]
not_valid_beta

[]

Beta distributions seems to be OK

### Creating a new impact methodology

In [12]:
def impact_method_create(impact_method_name,impact_method_metadata,impact_method_CF_list):
    
    impact_method=bw.Method(impact_method_name)
    impact_method.register(**impact_method_metadata)
    impact_method.write(impact_method_CF_list)
    
    return impact_method;

In [13]:
IW_methods=[method for method in bw.methods if 'IMPACTWorld+' in str(method)]
IW_methods

[('IMPACTWorld+ - Endpoint - only with spatial variability',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration',
  'Ecosystem Quality',
  'Freshwater eutrophication, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, no uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability',
  'Ecosystem Quality',
  'Terrestrial acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability',
  'Ecosystem Quality',
  'Land occupation, biodiversity, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability',
  'Human Health',
  'Water availability, human health, GLO, with uncert'),
 

In [14]:
unit_to_ic_names={'DALY':'Human Health','PDF.m2.yr':'Ecosystem Quality'}

impact_methodology_name='IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017'

# Get all impact category names and metadata from olca dictionary 
midpoint_name_list=[]
endpoint_name_list=[]
impact_cat_name_list=[]
impact_cat_metadata_dict={}
CF_list=[]

for cf in iw_olca_uncert_dict:
    
    # Create all impact category names
    midpoint_name=iw_olca_uncert_dict[cf]['impact_cat_name']
    endpoint_name=unit_to_ic_names[iw_olca_uncert_dict[cf]['CF_unit']]
    impact_cat_name=(impact_methodology_name,endpoint_name,midpoint_name)
    midpoint_name_list=list(set(midpoint_name_list)|set([midpoint_name]))
    endpoint_name_list=list(set(endpoint_name_list)|set([endpoint_name]))
    impact_cat_name_list=list(set(impact_cat_name_list)|set([impact_cat_name]))
    
    # Create metadata for all impact category names
    impact_cat_metadata={'description':'IMPACT World+ version as in openLCA, global default version, endpoint, only includes regionalized impact categories, generalized beta distributions as uncertainty when possible',
                          'filename': 'All IC regionalized_with uncertainty.xls',
                          'CF unit': iw_olca_uncert_dict[cf]['CF_unit'],
                          'Per elementary flow unit': iw_olca_uncert_dict[cf]['CF_per_unit_name']}   
    impact_cat_metadata_dict[impact_cat_name]=impact_cat_metadata
    
    # Create list of CF for all impact category names
    CF={('biosphere3', iw_olca_uncert_dict[cf]['ef_UUID']):(impact_cat_name,iw_uncert_dict_beta[cf])}
    CF_list.append(CF)

CF_dict={}

for ic in impact_cat_name_list:
    CF_dict[ic]=[]


for cf in CF_list:
    for ef in cf:
        ef_to_add=(ef,cf[ef][1])
        CF_dict[cf[ef][0]].append(ef_to_add)
        

CF_dict

{('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, with uncert'): [(('biosphere3',
    '4841a0fe-c250-4a39-94a1-1bc31426c0f1'),
   {'amount': 0.799194406,
    'loc': 0.336155493,
    'maximum': 93.83769727,
    'minimum': 0.002406182,
    'shape': 39.24918627,
    'uncertainty type': 13,
    'uncertainty_type': 13}),
  (('biosphere3', '9b00c244-fcce-4d38-922b-601a81f2ba34'),
   {'amount': 1.591727031,
    'loc': 0.05646066,
    'maximum': 146.2778145,
    'minimum': 0.001710379,
    'shape': 5.137727289,
    'uncertainty type': 13,
    'uncertainty_type': 13}),
  (('biosphere3', 'e1d913d8-214c-4957-b3e5-36ccc50f3724'),
   {'amount': 1.746893022,
    'loc': 0.074258037,
    'maximum': 1556.719119,
    'minimum': 0.000401678,
    'shape': 66.11494813,
    'uncertainty type': 13,
    'uncertainty_type': 13}),
  (('biosphere3', 'd9945575-a965-4bf3-9cd7-584263172abd'),


In [15]:
for ic in impact_cat_name_list:
    impact_method_name=ic
    impact_method_metadata=impact_cat_metadata_dict[ic]
    impact_method_CF=CF_dict[ic]

    impact_method_create(impact_method_name,impact_method_metadata,impact_method_CF)

### Load impact categories

In [26]:
IW_with_uncert_beta=[m for m in bw.methods 
                     if 'IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017' in str(m)]
IW_with_uncert_beta_wo=[m for m in IW_with_uncert_beta 
                             if 'Land transformation' not in str(m) 
                             and 'Land occupation' not in str(m)
                             and 'Freshwater eutrophication' not in str(m)]
IW_with_uncert_beta

[('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Human Health',
  'Water availability, human health, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Terrestrial acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land occupation, biodiversity, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only wit

In [17]:
bw.Method(IW_with_uncert_beta[0]).load()

[(('biosphere3', '4841a0fe-c250-4a39-94a1-1bc31426c0f1'),
  {'amount': 0.799194406,
   'loc': 0.336155493,
   'maximum': 93.83769727,
   'minimum': 0.002406182,
   'shape': 39.24918627,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', '9b00c244-fcce-4d38-922b-601a81f2ba34'),
  {'amount': 1.591727031,
   'loc': 0.05646066,
   'maximum': 146.2778145,
   'minimum': 0.001710379,
   'shape': 5.137727289,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', 'e1d913d8-214c-4957-b3e5-36ccc50f3724'),
  {'amount': 1.746893022,
   'loc': 0.074258037,
   'maximum': 1556.719119,
   'minimum': 0.000401678,
   'shape': 66.11494813,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', 'd9945575-a965-4bf3-9cd7-584263172abd'),
  {'amount': 0.799194406,
   'loc': 0.336155493,
   'maximum': 93.83769727,
   'minimum': 0.002406182,
   'shape': 39.24918627,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', 'e8c30bb3-92cc-4

## Testing the impact methods with uncertainty

Function for basic LCA calculation for multiple impact categories and multiple activities

In [18]:
#setup_name='all elementary flows with IW'
#list_fu_tuple = [(act,1)]
#list_methods_name = IW

def calc_MultiLCA(setup_name,list_fu_tuple,list_methods_name):
    
    list_functional_units_dict = [{act[0].key:act[1]} for act in list_fu_tuple]
    
    bw.calculation_setups[setup_name] = {'inv':list_functional_units_dict, 'ia':list_methods_name}
    myMultiLCA = bw.MultiLCA(setup_name)
    myMultiLCA.results.shape
    myMultiLCA.results
    
    act_names_list=[act[0]['name'] for act in list_fu_tuple]
    
    return pd.DataFrame(index=list_methods_name, columns=[act_names_list], data=myMultiLCA.results.T);

Function for Monte with multiple impact categories, 1 activity

In [19]:
#list_methods_name = IW_with_uncert
#fu_tuple = (act,1)
#iterations = 10

def MC_multi_impact(fu_tuple,list_methods_name,iterations):
    
    MC_results_dict={}
    fu_dict={fu_tuple[0]:fu_tuple[1]}
    
    for ic in list_methods_name:
        print(ic)
        MC_results_per_IC = bw.MonteCarloLCA(fu_dict,ic)
        scores = [next(MC_results_per_IC) for _ in range(iterations)]
        MC_results_dict[ic]=scores
    
    return MC_results_dict;

Function for Monte Carlo, one impact category, one activity

In [20]:
def MC_one_IC(imp_cat_name,act,fu,iterations):
    myFirstMonteCarlo = bw.MonteCarloLCA({act:fu},imp_cat_name)
    scores = [next(myFirstMonteCarlo) for _ in range(iterations)]
    return scores,

### Test on an activity with all elementary flows

Create a process with all elementary flows with quantity = 1

In [ ]:
DB_test=bw.Database("For test")

all_ef_act=DB_test.new_activity("All elementary flows")
all_ef_act['name']="All elementary flows"
all_ef_act['exchanges']=ef_exc_list
all_ef_act.save()

In [ ]:
elem_flow_DB=bw.Database('biosphere3')

all_ef_act=DB_test.get("All elementary flows")

for ef in elem_flow_DB:
    new_exc=all_ef_act.new_exchange(amount=1.0,input= ef.key,type='biosphere')
    new_exc.save()
    
all_ef_act.save()

Test on basic LCA calculation

In [21]:
DB_test=bw.Database("For test")
act=DB_test.get("All elementary flows")

setup_name='all elementary flows with IW - generalized beta'
list_fu_tuple = [(act,1)]
list_methods_name = IW_with_uncert_beta

calc_MultiLCA(setup_name,list_fu_tuple,list_methods_name)


,All elementary flows
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Ecosystem Quality, Freshwater acidification, GLO, with uncert)",2.029981e+01
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Human Health, Water availability, human health, GLO, with uncert)",2.258591e-04
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Ecosystem Quality, Terrestrial acidification, GLO, with uncert)",1.397056e+02
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Ecosystem Quality, Land occupation, biodiversity, GLO, with uncert)",4.033116e+00
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Ecosystem Quality, Land transformation, biodiversity, GLO, with uncert)",-8.881784e-15
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Ecosystem Quality, Freshwater eutrophication, GLO, with uncert)",9.912974e+00
"(IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017, Ecosystem Quality, Marine eutrophication, GLO, with uncert)",3.463642e+01


Test on Monte Carlo simulation

In [24]:
DB_test=bw.Database("For test")
act=DB_test.get("All elementary flows")

list_methods_name = IW_with_uncert_beta
fu_tuple = (act,1)
iterations = 10

scores=MC_multi_impact(fu_tuple,list_methods_name,iterations)
scores

('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Land occupation, biodiversity, GLO, with uncert')


MaximumIterationsError: 

Test on a process from ecoinvent without uncertainty

In [28]:
DB_ei_wo_uncert=bw.Database("ecoinvent 3.3 cutoff without uncertainty")
act_ei_random=DB_ei_wo_uncert.random()

In [29]:
DB_test=DB_ei_wo_uncert
act=act_ei_random

list_methods_name = IW_with_uncert_beta
fu_tuple = (act,1)
iterations = 10

scores_ei=MC_multi_impact(fu_tuple,list_methods_name,iterations)
scores_ei

('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Land occupation, biodiversity, GLO, with uncert')


MaximumIterationsError: 

#### Test without impact categories with errors

Land occupation, land transformation and freshwater eutrophication raise errors in MC simulation

In [30]:
DB_test=bw.Database("For test")
act=DB_test.get("All elementary flows")

list_methods_name = IW_with_uncert_beta_wo
fu_tuple = (act,1)
iterations = 10

scores=MC_multi_impact(fu_tuple,list_methods_name,iterations)
scores

('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, with uncert')


{('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, with uncert'): [9.732857998128113,
  19.5668003994529,
  87.3698870882834,
  24.391032749223022,
  43.25993314920798,
  16.144140438587332,
  19.396481048294582,
  35.83030218677692,
  36.16697899155949,
  24.86826970677656],
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Marine eutrophication, GLO, with uncert'): [34.00028986318351,
  34.993810791134976,
  34.20868612270355,
  34.404448125106356,
  34.4606351435589,
  33.8893123029805,
  33.77773181632264,
  35.71417214131218,
  36.209399643410705,
  34.21702922433585],
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Terrestrial acidification, GLO, with uncert'): [132.58485662

#### Test to identify the CF responsible for the error with bounded_random_variables

When maximum_iterations=50 --> 818 CF / 997 CF are errors [
'Land transformation, biodiversity, GLO, with uncert',
 'Freshwater eutrophication, GLO, with uncert',
 'Land occupation, biodiversity, GLO, with uncert',
 'Marine eutrophication, GLO, with uncert',
 'Terrestrial acidification, GLO, with uncert',
 'Freshwater acidification, GLO, with uncert',
 'Water availability, human health, GLO, with uncert'] 
 
 --> errors in all impact categories, except for CF without uncertainty!!!
 
 But Monte Carlo works for more CF...

In [31]:
error_in_MC={}
not_error_in_MC={}

for cf in iw_uncert_dict_beta:
    
    if type(iw_uncert_dict_beta[cf])== dict:
    
        uncert_dict=iw_uncert_dict_beta[cf]
        uncert=stats_arrays.UncertaintyBase.from_dicts(uncert_dict)
        

        try:
            stats_arrays.UncertaintyBase.bounded_random_variables(uncert,size=1,maximum_iterations=50)
            not_error_in_MC[cf]=iw_uncert_dict_beta[cf]

        except:
            print("error in "+ cf[0])
            error_in_MC[cf]=iw_uncert_dict_beta[cf]
    
    
error_in_MC
    
        

error in Freshwater acidification, GLO, with uncert
error in Freshwater eutrophication, GLO, with uncert
error in Terrestrial acidification, GLO, with uncert
error in Land transformation, biodiversity, GLO, with uncert
error in Land occupation, biodiversity, GLO, with uncert
error in Land occupation, biodiversity, GLO, with uncert
error in Freshwater acidification, GLO, with uncert
error in Land transformation, biodiversity, GLO, with uncert
error in Freshwater acidification, GLO, with uncert
error in Water availability, human health, GLO, with uncert
error in Land transformation, biodiversity, GLO, with uncert
error in Land occupation, biodiversity, GLO, with uncert
error in Land occupation, biodiversity, GLO, with uncert
error in Land transformation, biodiversity, GLO, with uncert
error in Land occupation, biodiversity, GLO, with uncert
error in Land occupation, biodiversity, GLO, with uncert
error in Land transformation, biodiversity, GLO, with uncert
error in Land transformation, b

{('Freshwater acidification, GLO, with uncert',
  '003697f1-54e9-409e-8179-23a95e99e4d7'): {'amount': 0.799194406,
  'loc': 0.336155493,
  'maximum': 93.83769727,
  'minimum': 0.002406182,
  'shape': 39.24918627,
  'uncertainty type': 13,
  'uncertainty_type': 13},
 ('Freshwater acidification, GLO, with uncert',
  '006aa3f7-59ba-450f-aa45-a2b2d1752647'): {'amount': 1.591727031,
  'loc': 0.05646066,
  'maximum': 146.2778145,
  'minimum': 0.001710379,
  'shape': 5.137727289,
  'uncertainty type': 13,
  'uncertainty_type': 13},
 ('Freshwater acidification, GLO, with uncert',
  '08a91e70-3ddc-11dd-954d-0050c2490048'): {'amount': 1.746893022,
  'loc': 0.074258037,
  'maximum': 1556.719119,
  'minimum': 0.000401678,
  'shape': 66.11494813,
  'uncertainty type': 13,
  'uncertainty_type': 13},
 ('Freshwater acidification, GLO, with uncert',
  '08a91e70-3ddc-11dd-96ee-0050c2490048'): {'amount': 0.799194406,
  'loc': 0.336155493,
  'maximum': 93.83769727,
  'minimum': 0.002406182,
  'shape': 39.

In [32]:
len(error_in_MC), len(not_error_in_MC)

(818, 0)

In [8]:
len(iw_uncert_dict_beta)

997

In [9]:
list_ic_error=[]

for cf in error_in_MC:
    list_ic_error=list(set(list_ic_error)|set([cf[0]]))
    
list_ic_error

['Land transformation, biodiversity, GLO, with uncert',
 'Freshwater eutrophication, GLO, with uncert',
 'Land occupation, biodiversity, GLO, with uncert',
 'Marine eutrophication, GLO, with uncert',
 'Terrestrial acidification, GLO, with uncert',
 'Freshwater acidification, GLO, with uncert',
 'Water availability, human health, GLO, with uncert']

In [25]:
stats_arrays.UncertaintyBase.bounded_random_variables(uncert,size=10)

NotImplementedError: 

Let's investigate each impact category with errors

Starting with freshwater europhication

First let's create an impact category for each elementary in this impact category, to test each CF

In [33]:
iw_FreshEutro=[ic for ic in IW_with_uncert_beta if 'Freshwater eutrophication' in str(ic)][0]
ic_FreshEutro=bw.Method(iw_FreshEutro)
iw_FreshEutro

('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
 'Ecosystem Quality',
 'Freshwater eutrophication, GLO, with uncert')

In [34]:
ic_FreshEutro=bw.Method(iw_FreshEutro)
impact_method_name_1="Single CF test for IW+ - test 2"
impact_method_name_2=iw_FreshEutro[2]

In [35]:
ic_FreshEutro=bw.Method(iw_FreshEutro)
impact_method_name_1="Single CF test for IW+ - test 2"
impact_method_name_2=iw_FreshEutro[2]

for cf in ic_FreshEutro.load():
    impact_method_CF_list=[cf]
    impact_method_name=(impact_method_name_1,impact_method_name_2,cf[0][1])
    impact_method_metadata={'description':'Test with single CF, IMPACT World+ version as in openLCA, global default version, endpoint, only includes regionalized impact categories, generalized beta distributions as uncertainty when possible',
                          'filename': 'All IC regionalized_with uncertainty.xls',
                          'CF unit': ic_FreshEutro.metadata.get('CF unit'),
                          'Per elementary flow unit': ic_FreshEutro.metadata.get('Per elementary flow unit')}
    impact_method_create(impact_method_name,impact_method_metadata,impact_method_CF_list)



In [ ]:
ic_FreshEutro=bw.Method(iw_FreshEutro)
impact_method_name_1="Double CF test for IW+ - test 2"
impact_method_name_2=iw_FreshEutro[2]

for cf in ic_FreshEutro.load():
    impact_method_CF_list=[cf]
    impact_method_name=(impact_method_name_1,impact_method_name_2,cf[0][1])
    impact_method_metadata={'description':'Test with single CF, IMPACT World+ version as in openLCA, global default version, endpoint, only includes regionalized impact categories, generalized beta distributions as uncertainty when possible',
                          'filename': 'All IC regionalized_with uncertainty.xls',
                          'CF unit': ic_FreshEutro.metadata.get('CF unit'),
                          'Per elementary flow unit': ic_FreshEutro.metadata.get('Per elementary flow unit')}
    impact_method_create(impact_method_name,impact_method_metadata,impact_method_CF_list)



In [36]:
all_ic_singleCF_FreshEutro=[ic for ic in bw.methods if impact_method_name_1 in str(ic) and impact_method_name_2 in str(ic)]
all_ic_singleCF_FreshEutro

[('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '2d4b8ec1-8d53-4e62-8a11-ebc45909b02e'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '490b267b-f429-4d9a-ac79-224e37fb4d58'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '6054fc06-c63a-48f2-bb80-3733834167bc'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '448934cb-8be4-4ae2-acdb-de9dec10459b'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  'b2631209-8374-431e-b7d5-56c96c6b6d79'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '2a89246a-8408-4775-a6db-7c33656f1469'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  'b1fca66f-8e83-469a-a7b5-018e14d5d545'),
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  'dc2458f6

In [37]:
DB_test=bw.Database("For test")
act=DB_test.get("All elementary flows")

list_methods_name = all_ic_singleCF_FreshEutro
fu_tuple = (act,1)
iterations = 20

MC_error_FreshEutro=[]
MC_results_dict={}
fu_dict={fu_tuple[0]:fu_tuple[1]}

for ic in list_methods_name:
    MC_results_per_IC = bw.MonteCarloLCA(fu_dict,ic)
    
    try:
        scores = [next(MC_results_per_IC) for _ in range(iterations)]
        MC_results_dict[ic]=scores
    except:
        MC_error_FreshEutro.append(ic)

In [38]:
MC_error_FreshEutro

[]

In [39]:
MC_results_dict

{('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '0857999c-aaf4-41f9-b3cc-5cc56909ecaa'): [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncert',
  '1727b41d-377e-43cd-bc01-9eaba946eccb'): [1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06,
  1.8624100448505487e-06],
 ('Single CF test for IW+ - test 2',
  'Freshwater eutrophication, GLO, with uncer

In [40]:
len(MC_results_dict),len(all_ic_singleCF_FreshEutro)

(54, 54)

In [41]:
one_cf=[ic for ic in all_ic_singleCF_FreshEutro if 'c8791f3c-3c4a-4278-91c0-483797d14da2' in str(ic)][0]
bw.Method(one_cf).load()

[(('biosphere3', 'c8791f3c-3c4a-4278-91c0-483797d14da2'),
  {'amount': 3.623163017,
   'loc': 0.000441333,
   'maximum': 48351.00323,
   'minimum': 5.58723e-06,
   'shape': 5.889147648,
   'uncertainty type': 13,
   'uncertainty_type': 13})]

In [42]:
all_CF_FreshEutro=bw.Method(iw_FreshEutro).load()
all_CF_FreshEutro

[(('biosphere3', 'fc0b5c85-3b49-42c2-a3fd-db7e57b696e3'),
  {'amount': 0.079709586,
   'loc': 0.000441333,
   'maximum': 1063.722071,
   'minimum': 1.22919e-07,
   'shape': 5.889147648,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', 'c8a80a68-83a7-4453-97f4-082ee1b60c71'), 0.0),
 (('biosphere3', 'f012d2d5-ebb4-4531-8285-0b98af3dcbd4'), 0.0),
 (('biosphere3', '4ab486b5-7102-4a88-a7fd-3e7b42e0e2e2'),
  {'amount': 11.08687883,
   'loc': 0.000441333,
   'maximum': 147954.0699,
   'minimum': 1.70969e-05,
   'shape': 5.889147648,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', '503ccbc4-8697-46e7-9a9b-beee17d4cdec'),
  {'amount': 3.623163017,
   'loc': 0.000441333,
   'maximum': 48351.00323,
   'minimum': 5.58723e-06,
   'shape': 5.889147648,
   'uncertainty type': 13,
   'uncertainty_type': 13}),
 (('biosphere3', '4574cd53-77cc-4540-827b-5736dda9c4d7'),
  {'amount': 0.079709586,
   'loc': 0.000441333,
   'maximum': 1063.722071,
   'minimum'

In [43]:
len(all_CF_FreshEutro)

54

In [48]:
for prod in act.production():
    print(str(prod))

Exchange: 1.0 None 'All elementary flows' (None, GLO, None) to 'All elementary flows' (None, GLO, None)>
Exchange: 1.0 None 'All elementary flows' (None, GLO, None) to 'All elementary flows' (None, GLO, None)>
Exchange: 1.0 None 'All elementary flows' (None, GLO, None) to 'All elementary flows' (None, GLO, None)>


Check which CF are really used --> not all CF are used (which makes sense because ef ID are from openLCA)

In [44]:
act=DB_test.get("All elementary flows")
act_exc=[]
for exc in act.exchanges():
    act_exc.append(exc.input.key[1])
    
act_exc

['b66104c3-8096-4c8d-abbc-cae04770d8d1',
 '87a69874-34f9-4070-a525-aa301ab2634b',
 '98313bb5-3f35-4e11-bf3e-03ad2dae591b',
 '7e59174a-d66b-4304-81dc-e0aeb3c58ae2',
 'fc1e35a8-88d4-4baf-a41a-563af2af9174',
 'f7882aae-499b-42ab-8fa0-b4e20b609cc0',
 'fd9e2be1-a651-4c40-b0bb-e7daa4f56ad4',
 'ccc02a20-bda2-4862-bb95-2da1db7ad49e',
 'd0aaa60f-40d9-4c12-a486-954333aa549f',
 'e7c5db65-775c-4ada-8a63-32c67792bd52',
 '834ffc71-e8a1-4b2d-b960-8a924c708954',
 '046887e9-5de9-44c0-8461-3c6da8e9e4b1',
 'c317b216-2668-40f8-9e24-1ce183d53697',
 'f3f34697-7231-4575-aaf5-e35b972d329a',
 'bbff44a2-cd6d-4200-98c7-e4f772fd24c5',
 '38eff837-5465-47a9-a1c9-e1edd70922ef',
 '4c0d236e-e50f-409e-b84b-b053da3394fa',
 'ec758e92-f235-4372-85b6-40a212e6f86c',
 'd7e80dce-7d72-4e93-88c0-57fb67fc9ab9',
 '3afbc2d8-1b51-4d43-87a7-e68291c1e999',
 '873d28d3-e53b-46a6-aaa0-63b923f0a56c',
 'ef4884eb-c75d-4ef8-9170-11b20401cbca',
 '3681c873-199a-45ec-9521-81e430bed4df',
 '8d0ca1a5-6a40-4eda-861a-a0fccdc81b19',
 'ae521be9-6534-

In [45]:
FreshEutro_exc=[]

for cf in all_ic_singleCF_FreshEutro:
    FreshEutro_exc.append(cf[2])
    
FreshEutro_exc

['2d4b8ec1-8d53-4e62-8a11-ebc45909b02e',
 '490b267b-f429-4d9a-ac79-224e37fb4d58',
 '6054fc06-c63a-48f2-bb80-3733834167bc',
 '448934cb-8be4-4ae2-acdb-de9dec10459b',
 'b2631209-8374-431e-b7d5-56c96c6b6d79',
 '2a89246a-8408-4775-a6db-7c33656f1469',
 'b1fca66f-8e83-469a-a7b5-018e14d5d545',
 'dc2458f6-30fc-4c71-97f6-ff4c5470bd3a',
 '18e1aef2-00e7-410d-9613-fdfb305a689e',
 'aedea697-930a-4a0e-9e4f-b9c369aa9a33',
 '4ba931a8-50e1-4b94-abe2-f645e93bb574',
 '3e129646-13b3-45a9-840a-c6b499f654b9',
 'c8a80a68-83a7-4453-97f4-082ee1b60c71',
 '8d33cd5f-c365-44b8-bb70-a211d5bb15dd',
 '43156184-3040-4095-9085-38d8895406cc',
 '0857999c-aaf4-41f9-b3cc-5cc56909ecaa',
 'dcd473f2-93b3-416b-8b97-d157b6ce1fe8',
 '5687da3e-fa56-4ad0-81bc-a5f6d175433e',
 'fc0b5c85-3b49-42c2-a3fd-db7e57b696e3',
 '70d467b6-115e-43c5-add2-441de9411348',
 'a8fea8cf-9b09-49fc-aa58-5a732494a2a0',
 '961ccd56-2eee-4f57-bb18-fc4362b49000',
 'ad62c648-1af8-4805-bbfb-64c6361eb9f6',
 '6a7e5863-714c-4dd4-9fbc-0d92b8f67e93',
 'd07a6bba-3816-

In [46]:
act_exc_set=set(act_exc)
in_exc=[]
not_in_exc=[]

for ef in FreshEutro_exc:
    if ef in act_exc_set:
        in_exc.append(ef)
    else:
        not_in_exc.append(ef)

len(in_exc), len(not_in_exc)

(18, 36)

In [47]:
imp_cat_name=iw_FreshEutro
act=DB_test.get("All elementary flows")
fu=1
iterations=10

score_FreshEutro_all=MC_one_IC(imp_cat_name,act,fu,iterations)
score_FreshEutro_all

MaximumIterationsError: 

Let's try to remove CF with ID that do not exist in Brightway

In [44]:
in_exc

['1727b41d-377e-43cd-bc01-9eaba946eccb',
 'fc0b5c85-3b49-42c2-a3fd-db7e57b696e3',
 '2d4b8ec1-8d53-4e62-8a11-ebc45909b02e',
 '329fc7d8-4011-4327-84e4-34ff76f0e42d',
 '70d467b6-115e-43c5-add2-441de9411348',
 'f012d2d5-ebb4-4531-8285-0b98af3dcbd4',
 '961ccd56-2eee-4f57-bb18-fc4362b49000',
 'be10dd54-eb4d-48ae-9e14-7d79ce8b170d',
 '490b267b-f429-4d9a-ac79-224e37fb4d58',
 'c8791f3c-3c4a-4278-91c0-483797d14da2',
 'ec76fb5c-b402-416b-825c-53e074fe7525',
 'aedea697-930a-4a0e-9e4f-b9c369aa9a33',
 'a8fea8cf-9b09-49fc-aa58-5a732494a2a0',
 'b1fca66f-8e83-469a-a7b5-018e14d5d545',
 '76b09afe-52b9-4ecc-b97a-00b49ad8ed1a',
 '91955aba-6ebc-4413-9342-f0298860b4aa',
 '57c27ba7-a61a-4932-8cd5-095c88ff0245',
 'b2631209-8374-431e-b7d5-56c96c6b6d79']

In [53]:
in_exc_set=set(in_exc)
CFs=bw.Method(iw_FreshEutro)

new_CF_list=[]

for cf in CFs.load():
    if cf[0][1] in in_exc_set:
        new_CF_list.append(cf)

impact_method_CF_list=new_CF_list
impact_method_name=('IMPACTWorld+ - Endpoint - spatial var - Generalized beta - only with elementary flows existing in BW2',
                    iw_FreshEutro[1],iw_FreshEutro[2])
impact_method_metadata={'description':'IMPACT World+ only with elementary flows existing in BW2',
                          'filename': 'All IC regionalized_with uncertainty.xls',
                          'CF unit': CFs.metadata.get('CF unit'),
                          'Per elementary flow unit': CFs.metadata.get('Per elementary flow unit')}
impact_method_create(impact_method_name,impact_method_metadata,impact_method_CF_list)
    


Brightway2 Method: IMPACTWorld+ - Endpoint - spatial var - Generalized beta - only with elementary flows existing in BW2: Ecosystem Quality: Freshwater eutrophication, GLO, with uncert

In [50]:
 new_CF_list

[(('biosphere3', '1727b41d-377e-43cd-bc01-9eaba946eccb'),
  {'amount': 3.623163017,
   'loc': 0.000441333,
   'maximum': 48351.00323,
   'minimum': 5.58723e-06,
   'shape': 5.889147648,
   'uncertainty type': 13}),
 (('biosphere3', 'b1fca66f-8e83-469a-a7b5-018e14d5d545'), 0.0),
 (('biosphere3', 'aedea697-930a-4a0e-9e4f-b9c369aa9a33'), 0.0),
 (('biosphere3', 'a8fea8cf-9b09-49fc-aa58-5a732494a2a0'), 0.0),
 (('biosphere3', 'f012d2d5-ebb4-4531-8285-0b98af3dcbd4'), 0.0),
 (('biosphere3', '57c27ba7-a61a-4932-8cd5-095c88ff0245'), 0.0),
 (('biosphere3', '70d467b6-115e-43c5-add2-441de9411348'),
  {'amount': 0.079709586,
   'loc': 0.000441333,
   'maximum': 1063.722071,
   'minimum': 1.22919e-07,
   'shape': 5.889147648,
   'uncertainty type': 13}),
 (('biosphere3', '490b267b-f429-4d9a-ac79-224e37fb4d58'), 0.0),
 (('biosphere3', 'be10dd54-eb4d-48ae-9e14-7d79ce8b170d'), 0.0),
 (('biosphere3', '329fc7d8-4011-4327-84e4-34ff76f0e42d'), 0.0),
 (('biosphere3', 'ec76fb5c-b402-416b-825c-53e074fe7525'), 

In [56]:
impact_method_name

('IMPACTWorld+ - Endpoint - spatial var - Generalized beta - only with elementary flows existing in BW2',
 'Ecosystem Quality',
 'Freshwater eutrophication, GLO, with uncert')

In [62]:
iw_FreshEutro_only_bw=[ic for ic in bw.methods if 'Freshwater eutrophication' in str(ic) 
                       and 'IMPACTWorld+ - Endpoint - spatial var - Generalized beta - only with elementary flows existing in BW2' in str(ic)][0]
ic_FreshEutro_only_bw=bw.Method(iw_FreshEutro)
iw_FreshEutro_only_bw

('IMPACTWorld+ - Endpoint - spatial var - Generalized beta - only with elementary flows existing in BW2',
 'Ecosystem Quality',
 'Freshwater eutrophication, GLO, with uncert')

In [63]:
imp_cat_name=iw_FreshEutro_only_bw
act=DB_test.get("All elementary flows")
fu=1
iterations=10

score_FreshEutro_all=MC_one_IC(imp_cat_name,act,fu,iterations)
score_FreshEutro_all

MaximumIterationsError: 

Still maximum iterations error, even without ef not defined in BW2....

Let's investigate